In [2]:
# Importing dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

test_df = pd.read_csv('../Resources/test_clean.csv')
train_df = pd.read_csv('../Resources/train_clean.csv')
survivors_df = pd.read_csv('../Resources/gender_submission.csv')

display(test_df)
display(train_df)
display(survivors_df)

,PassengerId,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,892,3,Kelly,Mr James,male,34.5,0,0,330911,7.8292,Q,"D,E,F,G"
1,893,3,Wilkes,Mrs James (Ellen Needs),female,47.0,1,0,363272,7.0000,S,"D,E,F,G"
2,894,2,Myles,Mr Thomas Francis,male,62.0,0,0,240276,9.6875,Q,"D,E,F,G"
3,895,3,Wirz,Mr Albert,male,27.0,0,0,315154,8.6625,S,"D,E,F,G"
4,896,3,Hirvonen,Mrs Alexander (Helga E Lindqvist),female,22.0,1,1,3101298,12.2875,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,Spector,Mr Woolf,male,23.0,0,0,A.5. 3236,8.0500,S,"D,E,F,G"
414,1306,1,Oliva y Ocana,Dona Fermina,female,39.0,0,0,PC 17758,108.9000,C,"A,B,C,D,E"
415,1307,3,Saether,Mr Simon Sivertsen,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S,"D,E,F,G"
416,1308,3,Ware,Mr Frederick,male,34.0,0,0,359309,8.0500,S,"D,E,F,G"


,PassengerId,Pclass,last_name,first_name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,1,3,Braund,Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,S,"D,E,F,G"
1,2,1,Cumings,Mrs John Bradley (Florence Briggs Thayer),female,38.0,1,0,PC 17599,71.2833,C,"A,B,C,D,E"
2,3,3,Heikkinen,Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,S,"D,E,F,G"
3,4,1,Futrelle,Mrs Jacques Heath (Lily May Peel),female,35.0,1,0,113803,53.1000,S,"A,B,C,D,E"
4,5,3,Allen,Mr William Henry,male,35.0,0,0,373450,8.0500,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...,...,...,...
884,887,2,Montvila,Rev Juozas,male,27.0,0,0,211536,13.0000,S,"D,E,F,G"
885,888,1,Graham,Miss Margaret Edith,female,19.0,0,0,112053,30.0000,S,"A,B,C,D,E"
886,889,3,Johnston,Miss Catherine Helen Carrie,female,7.0,1,2,W./C. 6607,23.4500,S,"D,E,F,G"
887,890,1,Behr,Mr Karl Howell,male,26.0,0,0,111369,30.0000,C,"A,B,C,D,E"


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [3]:
# Dropping non-beneficial ID columns
test_df_dropped = test_df.drop(['PassengerId', 'last_name', 'first_name'], axis=1)
train_df_dropped = train_df.drop(['PassengerId', 'last_name', 'first_name'], axis=1)

display(test_df_dropped)
display(train_df_dropped)

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,3,male,34.5,0,0,330911,7.8292,Q,"D,E,F,G"
1,3,female,47.0,1,0,363272,7.0000,S,"D,E,F,G"
2,2,male,62.0,0,0,240276,9.6875,Q,"D,E,F,G"
3,3,male,27.0,0,0,315154,8.6625,S,"D,E,F,G"
4,3,female,22.0,1,1,3101298,12.2875,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...
413,3,male,23.0,0,0,A.5. 3236,8.0500,S,"D,E,F,G"
414,1,female,39.0,0,0,PC 17758,108.9000,C,"A,B,C,D,E"
415,3,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,S,"D,E,F,G"
416,3,male,34.0,0,0,359309,8.0500,S,"D,E,F,G"


,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck
0,3,male,22.0,1,0,A/5 21171,7.2500,S,"D,E,F,G"
1,1,female,38.0,1,0,PC 17599,71.2833,C,"A,B,C,D,E"
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S,"D,E,F,G"
3,1,female,35.0,1,0,113803,53.1000,S,"A,B,C,D,E"
4,3,male,35.0,0,0,373450,8.0500,S,"D,E,F,G"
...,...,...,...,...,...,...,...,...,...
884,2,male,27.0,0,0,211536,13.0000,S,"D,E,F,G"
885,1,female,19.0,0,0,112053,30.0000,S,"A,B,C,D,E"
886,3,female,7.0,1,2,W./C. 6607,23.4500,S,"D,E,F,G"
887,1,male,26.0,0,0,111369,30.0000,C,"A,B,C,D,E"


In [7]:
# Looking at the number of unique values in each column for test_df
print(test_df_dropped.nunique())

Pclass        3
Sex           2
Age          80
SibSp         7
Parch         8
Ticket      363
Fare        169
Embarked      3
Deck          2
dtype: int64


In [8]:
# Looking at the number of unique values in each column for train_df
print(train_df_dropped.nunique())

Pclass        3
Sex           2
Age          89
SibSp         7
Parch         7
Ticket      679
Fare        248
Embarked      3
Deck          2
dtype: int64


In [17]:
train_df_dropped['Ticket'].value_counts()

Ticket
1601        7
CA. 2343    7
347082      7
CA 2144     6
347088      6
           ..
9234        1
19988       1
2693        1
PC 17612    1
370376      1
Name: count, Length: 679, dtype: int64

In [18]:
train_dummies_df = pd.get_dummies(train_df_dropped)
train_dummies_df

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Ticket_110152,Ticket_110413,Ticket_110465,...,Ticket_W./C. 6608,Ticket_W./C. 6609,Ticket_W.E.P. 5734,Ticket_W/C 14208,Ticket_WE/P 5735,Embarked_C,Embarked_Q,Embarked_S,"Deck_A,B,C,D,E","Deck_D,E,F,G"
0,3,22.0,1,0,7.2500,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,True
1,1,38.0,1,0,71.2833,True,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
2,3,26.0,0,0,7.9250,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
3,1,35.0,1,0,53.1000,True,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
4,3,35.0,0,0,8.0500,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,2,27.0,0,0,13.0000,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,True
885,1,19.0,0,0,30.0000,True,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
886,3,7.0,1,2,23.4500,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
887,1,26.0,0,0,30.0000,False,True,False,False,False,...,False,False,False,False,False,True,False,False,True,False


In [ ]:
# Spltting preprocessed data into features and target arrays
X = train_dummies_df
y = survivors_df['Survived']

In [19]:
survivors_df['PassengerId'].values

array([ 892,  893,  894,  895,  896,  897,  898,  899,  900,  901,  902,
        903,  904,  905,  906,  907,  908,  909,  910,  911,  912,  913,
        914,  915,  916,  917,  918,  919,  920,  921,  922,  923,  924,
        925,  926,  927,  928,  929,  930,  931,  932,  933,  934,  935,
        936,  937,  938,  939,  940,  941,  942,  943,  944,  945,  946,
        947,  948,  949,  950,  951,  952,  953,  954,  955,  956,  957,
        958,  959,  960,  961,  962,  963,  964,  965,  966,  967,  968,
        969,  970,  971,  972,  973,  974,  975,  976,  977,  978,  979,
        980,  981,  982,  983,  984,  985,  986,  987,  988,  989,  990,
        991,  992,  993,  994,  995,  996,  997,  998,  999, 1000, 1001,
       1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012,
       1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023,
       1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034,
       1035, 1036, 1037, 1038, 1039, 1040, 1041, 10